In [2]:
# IMPORTS
import pandas as pd

import tkinter as tk
from tkinter import filedialog as fd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

import re
from unidecode import unidecode

from datetime import datetime

import bcrypt

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')

# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']

# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [4]:
# funcion para ejecutar una script SQL
# estado = 0 para select
# estado = 1 para update
def dbQuery(q, estado = 0):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            if estado==1:
                conn.commit()  # Commit the transaction
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [5]:
# GENERAR CODIGO SLUG, CODIGO UNICO
def abbreviate_para_slug(name):
    # Eliminar acentos y otros caracteres especiales
    name = unidecode(name)
    # Mantener solo caracteres alfanuméricos, espacios y guiones
    name = re.sub(r"[^\w\s-]", "", name)
    # Convertir a minúsculas
    name = name.lower()
    # Reemplazar espacios y guiones por un solo guión
    abbreviated = re.sub(r"[\s-]+", "-", name)
    return abbreviated.strip('-')

In [6]:
file_name = 'G:/OneDrive - Ingenio Azucarero Guabira S.A/AGRO-CITTCA/DATA PARA CARGAR/PROPIEDADES.XLSX'

In [7]:
data_lotes = pd.read_excel(file_name, sheet_name='DATA_LOTES')

In [8]:
data_lotes.columns

Index(['codigo', 'nombre', 'area', 'fs', 'soca', 'estado_lote', 'rend',
       'propiedad_id', 'usuario_id', 'variedad_id', 'textura_id', 'Columna2',
       'Columna1', 'unidad_01', 'unidad_02', 'unidad_03', 'unidad_04',
       'variedad', 'fs2', 'fsc23', 'textura', 'financia', 'cultivo', 'zona'],
      dtype='object')

In [9]:
lotes = data_lotes[['codigo', 'nombre', 'area', 'fs', 'soca', 'estado_lote', 'rend',
       'propiedad_id', 'usuario_id', 'variedad_id', 'textura_id']]

In [10]:
lotes

,codigo,nombre,area,fs,soca,estado_lote,rend,propiedad_id,usuario_id,variedad_id,textura_id
0,000020000L1,L1,0.80,20170920,5,3,57,361,18354,14,5
1,000040000L8,L8,1.22,0,122,4,57,362,1843,16,5
2,000040000L6,L6,1.33,0,122,4,57,362,1843,16,5
3,000040000L3,L3,2.41,0,122,4,57,362,1843,16,2
4,000040000L5,L5,0.48,0,122,4,57,362,1843,16,2
...,...,...,...,...,...,...,...,...,...,...,...
10279,000019910000L2,L2,3.25,20190623,3,3,57,1819,3473,14,5
10280,000019910000L3.1,L3.1,3.48,20190605,3,3,57,1819,3473,12,5
10281,000019910000L3.3,L3.3,2.10,20190605,3,3,57,1819,3473,12,5
10282,000019910000L4.2,L4.2,2.22,20180526,4,3,57,1819,3473,14,5


In [11]:
# seleccionar todas las prop sin lotes
select = 'SELECT p.* FROM propiedad p LEFT JOIN lote l ON p.id = l.propiedad_id WHERE l.propiedad_id IS NULL and p.periodo_id=15 and p.estado=\'activo\''
select_prop = dbQuery(select)
df_props_agro_sinlote = pd.DataFrame(select_prop)
df_props_agro_sinlote.head(3)

,id,codigo,nombre,slug,superficie_total,foto,estado,enlace,cuadrante,usuario_id,periodo_id,created_at,updated_at
0,424,118,118 - EL ARROZAL--FRITZER,118-el-arrozal-fritzer-424,129.38000000000002,/imagen/foto_propiedad.jpg,activo,None,359,613,15,2023-08-30 18:57:31,2023-08-30 18:57:31


In [12]:
select_prop

[{'id': 424,
  'codigo': '118',
  'nombre': '118 - EL ARROZAL--FRITZER',
  'slug': '118-el-arrozal-fritzer-424',
  'superficie_total': '129.38000000000002',
  'foto': '/imagen/foto_propiedad.jpg',
  'estado': 'activo',
  'enlace': None,
  'cuadrante': '359',
  'usuario_id': 613,
  'periodo_id': 15,
  'created_at': datetime.datetime(2023, 8, 30, 18, 57, 31),
  'updated_at': datetime.datetime(2023, 8, 30, 18, 57, 31)}]

In [13]:
lotes_cargar = list(set(df_props_agro_sinlote['id']))
len(lotes_cargar)

1

In [14]:
a = 1
for k in lotes_cargar:
    print('Inicio resgistro propiedad:', k)
    id_prop = k
    lotes_prop = lotes[lotes['propiedad_id']==id_prop]
    j = 1
    for i, r in lotes_prop.iterrows():
        insert_query = 'INSERT INTO lote\
        (codigo, nombre, slug, superficie, fecha_siembra, corte, estado_lote, rendimiento, estado, observacion, propiedad_id, usuario_id, variedad_id, textura_id, created_at, updated_at)\
        VALUES\
        (\'{codigo}\', \'{nombre}\', \'{slug}\', \'{superficie}\', \'{fecha_siembra}\', \'{corte}\', \'{estado_lote}\', \'{rendimiento}\', \'{estado}\', \'{observacion}\', {propiedad_id}, \'{usuario_id}\', {variedad_id}, {textura_id}, NOW(), NOW())'\
        .format(
            codigo=r['codigo'],
            nombre=r['nombre'],
            slug='',
            superficie=r['area'],
            fecha_siembra=r['fs'],
            corte=r['soca'],
            estado_lote=r['estado_lote'],
            rendimiento='57.0',
            estado='activo',
            observacion='',
            propiedad_id=r['propiedad_id'],
            usuario_id=r['usuario_id'],
            variedad_id=r['variedad_id'],
            textura_id=r['textura_id']
        )
        insert_row = dbQuery(insert_query, 1)
        select_reg = 'select max(id) as maximo from lote'
        select_reg = dbQuery(select_reg)
        id_max = select_reg[0]['maximo']
        slug = abbreviate_para_slug(r['nombre']) + '-' + str(id_max)
        update = 'update lote set slug=\'{sl}\' where id = {id_lote}'.format(sl=slug, id_lote = id_max)
        update_row = dbQuery(update, 1)
        print('Registro Numero:', j, ' - Se registro lote ID:', r['codigo'])
        j = j + 1
    print('Fin resgistro propiedad:', k, '=============================================================', a)

Inicio resgistro propiedad: 424
Registro Numero: 1  - Se registro lote ID: 00001180000L12
Registro Numero: 2  - Se registro lote ID: 00001180000L3.1
Registro Numero: 3  - Se registro lote ID: 00001180000L14.2
Registro Numero: 4  - Se registro lote ID: 00001180000L8.3
Registro Numero: 5  - Se registro lote ID: 00001180000L13
Registro Numero: 6  - Se registro lote ID: 00001180000L11
Registro Numero: 7  - Se registro lote ID: 00001180000L9.3
Registro Numero: 8  - Se registro lote ID: 00001180000L8.1
Registro Numero: 9  - Se registro lote ID: 00001180000L7
Registro Numero: 10  - Se registro lote ID: 00001180000L9.2
Registro Numero: 11  - Se registro lote ID: 00001180000L9.1
Registro Numero: 12  - Se registro lote ID: 00001180000L10
Registro Numero: 13  - Se registro lote ID: 00001180000L2
Registro Numero: 14  - Se registro lote ID: 00001180000L8.2
Registro Numero: 15  - Se registro lote ID: 00001180000L4_5
Registro Numero: 16  - Se registro lote ID: 00001180000L3.2
Registro Numero: 17  - S

In [72]:
id_prop = 379

In [73]:
lotes_prop = lotes[lotes['propiedad_id']==id_prop]
lotes_prop

,codigo,nombre,area,fs,soca,estado_lote,rend,propiedad_id,usuario_id,variedad_id,textura_id
286,0000380000L1,L1,1.43,20170908,5,3,57,379,12087,12,3
287,0000380000L3,L3,0.55,20170907,5,3,57,379,12087,12,3
288,0000380000L2,L2,1.49,20160710,6,3,57,379,12087,12,3


In [80]:
j = 1
for i, r in lotes_prop.iterrows():
    insert_query = 'INSERT INTO lote\
    (codigo, nombre, slug, superficie, fecha_siembra, corte, estado_lote, rendimiento, estado, observacion, propiedad_id, usuario_id, variedad_id, textura_id, created_at, updated_at)\
    VALUES\
    (\'{codigo}\', \'{nombre}\', \'{slug}\', \'{superficie}\', \'{fecha_siembra}\', \'{corte}\', \'{estado_lote}\', \'{rendimiento}\', \'{estado}\', \'{observacion}\', {propiedad_id}, \'{usuario_id}\', {variedad_id}, {textura_id}, NOW(), NOW())'\
    .format(
        codigo=r['codigo'],
        nombre=r['nombre'],
        slug='',
        superficie=r['area'],
        fecha_siembra=r['fs'],
        corte=r['soca'],
        estado_lote=r['estado_lote'],
        rendimiento='57.0',
        estado='activo',
        observacion='',
        propiedad_id=r['propiedad_id'],
        usuario_id=r['usuario_id'],
        variedad_id=r['variedad_id'],
        textura_id=r['textura_id']
    )
    insert_row = dbQuery(insert_query, 1)
    select_reg = 'select max(id) as maximo from lote'
    select_reg = dbQuery(select_reg)
    id_max = select_reg[0]['maximo']
    slug = abbreviate_para_slug(r['nombre']) + '-' + str(id_max)
    update = 'update lote set slug=\'{sl}\' where id = {id_lote}'.format(sl=slug, id_lote = id_max)
    update_row = dbQuery(update, 1)
    print('Registro Numero:', j, ' - Se registro lote ID:', r['codigo'])
    j = j + 1

Registro Numero: 1  - Se registro lote ID: 0000380000L1
Registro Numero: 2  - Se registro lote ID: 0000380000L3
Registro Numero: 3  - Se registro lote ID: 0000380000L2


In [47]:
insert_query

"INSERT INTO lote    (codigo, nombre, slug, superficie, fecha_siembra, corte, estado_lote, rendimiento, estado, observacion, propiedad_id, usuario_id, variedad_id, textura_id, created_at, updated_at)    VALUES    ('0000380000L2', 'L2', '', '1.49', '20160710', '6', '3', '57.0', 'activo', '', 379, '12087', 12, 3, NOW(), NOW())"

'l2'

In [56]:
select_reg = 'select max(id) as maximo from lote'
select_reg = dbQuery(select_reg)

6865